In [ ]:
import os
import MySQLdb  # Use MySQLdb instead of pytds for MySQL
import pandas as pd

# Connection parameters
server_name = ''
database_name = ''
user_name = ''
password = ''

# Establish a connection
conn = MySQLdb.connect(host=server_name, user=user_name, passwd=password, db=database_name)

# Create a cursor
cursor = conn.cursor()

# Execute a query to get all tables with their schema names
cursor.execute("""
    SELECT TABLE_SCHEMA, TABLE_NAME 
    FROM information_schema.tables 
    WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_SCHEMA = %s
""", (database_name,))

# Fetch all the tables with schema names
tables = cursor.fetchall()

# Create a folder for exporting tables
output_folder = f"./{database_name}_tables_export_datatype"
os.makedirs(output_folder, exist_ok=True)

# Create a list to store table schema information
schema_info = []

check_empty = True

# Function to get columns and their data types of a table
def get_table_columns(schema, table_name):
    cursor.execute("""
        SELECT COLUMN_NAME, DATA_TYPE
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = %s AND TABLE_NAME = %s
    """, (schema, table_name))
    return cursor.fetchall()

# Loop through each table and export to CSV
for schema, table_name in tables:
    try:
        # Get columns and their data types
        columns = get_table_columns(schema, table_name)
        for col_name, data_type in columns:
            schema_info.append({'table_name': f'{schema}.{table_name}', 'field_name': col_name, 'data_type': data_type})

        # Check if the CSV file already exists
        csv_filename = os.path.join(output_folder, f"{schema}__{table_name}.csv")
        if check_empty:
            # Check if the file is empty
            if os.path.exists(csv_filename):
                with open(csv_filename, 'r') as file:
                    # Read the first character to determine if the file is empty
                    first_char = file.read(1)
                    if not first_char:
                        # File is empty, proceed with export
                        print(f"Exporting '{table_name}' as '{csv_filename}' (including empty files).")
                    else:
                        # File is not empty, skip export
                        print(f"Skipped exporting '{table_name}' as '{csv_filename}' already exists and is not empty.")
                        continue
            else:
                # File does not exist, proceed with export
                print(f"Exporting '{table_name}' as '{csv_filename}' (including empty files).")
        else:
            if os.path.exists(csv_filename):
                # Skip export if file exists
                print(f"Skipped exporting '{table_name}' as '{csv_filename}' already exists.")
                continue

        # Execute a query to fetch all rows from the current table with the correct schema
        cursor.execute(f"SELECT * FROM {schema}.{table_name}")
        
        # Fetch all rows
        rows = cursor.fetchall()
        
        # Convert rows to a DataFrame
        df = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])

        # Export DataFrame to CSV
        df.to_csv(csv_filename, index=False)
        
        print(f"Table '{schema}.{table_name}' exported to '{csv_filename}'")
    
    except Exception as e:
        print(f"Error exporting table '{schema}.{table_name}': {e}")

# Convert schema_info to DataFrame and save to CSV
schema_df = pd.DataFrame(schema_info)
schema_df.to_csv(os.path.join(output_folder, f"{database_name}_schema_info.csv"), index=False)
print("Schema information exported to 'schema_info.csv'")

# Close the cursor and connection
cursor.close()
conn.close()
